In [1]:
import pandas as pd
import requests
import numpy as np

In [2]:
playlists = pd.read_csv("moyennes_playlists.csv")
playlists_no_na = playlists.dropna()
playlists_mean = playlists_no_na.iloc[:, 1:11].reset_index(drop=True)


playlists_med = playlists_no_na.iloc[:, 11:18].reset_index(drop=True)
# UTILISER LA MEDIANE 




In [161]:
playlists_med

,med_valence,med_danceability,med_energy,med_acousticness,med_liveness,med_speechiness,med_instrumentalness
0,0.5030,0.7010,0.6085,0.19700,0.1310,0.05700,0.000000
1,0.4570,0.7875,0.6135,0.07560,0.1315,0.23450,0.000000
2,0.5425,0.5955,0.7050,0.14450,0.1195,0.03505,0.000000
3,0.6215,0.7725,0.6860,0.13950,0.1185,0.09175,0.000008
4,0.4925,0.6640,0.5955,0.13350,0.1205,0.06460,0.000002
...,...,...,...,...,...,...,...
1390,0.4120,0.6460,0.5860,0.25750,0.1125,0.05495,0.000000
1391,0.5010,0.6835,0.6320,0.09410,0.1245,0.05750,0.000000
1392,0.4075,0.5695,0.5520,0.26050,0.1120,0.04325,0.000072
1393,0.6535,0.6870,0.7490,0.09685,0.1240,0.04775,0.000000


In [20]:
playlists_mean.iloc[:,3:]

,mean_valence,mean_danceability,mean_energy,mean_acousticness,mean_liveness,mean_speechiness,mean_instrumentalness
0,0.489742,0.683300,0.616320,0.257465,0.183326,0.092798,0.005109
1,0.489456,0.785720,0.602740,0.131570,0.146606,0.232856,0.012372
2,0.539462,0.590327,0.694712,0.203673,0.165504,0.043027,0.000105
3,0.618080,0.754340,0.682760,0.219143,0.163130,0.127918,0.017659
4,0.484911,0.665310,0.583040,0.251633,0.173775,0.123807,0.011088
...,...,...,...,...,...,...,...
1390,0.389423,0.624486,0.555957,0.364980,0.149979,0.072987,0.012603
1391,0.498406,0.676920,0.614670,0.205187,0.167980,0.096808,0.003083
1392,0.407217,0.597167,0.524167,0.383570,0.161173,0.076813,0.009128
1393,0.636093,0.676030,0.727190,0.143780,0.175988,0.084920,0.002124


In [196]:
def obj(med_valence, med_danceability, med_energy, med_acousticness, med_liveness, med_speechiness, med_instrumentalness):
    """
    Fixe les contraintes dans une liste"""
    objectif=[med_valence, med_danceability, med_energy, med_acousticness, med_liveness, med_speechiness, med_instrumentalness]
    return objectif


In [178]:
objectif_min = obj(0, 0, 0, 0, 0, 0, 0)
objectif_max = obj(0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8)

for i, j, k in zip(list(playlists_med.columns), objectif_min, objectif_max):
    playlists_med[str(i) + "_filtre"] = playlists_med[(playlists_med[i]>=j) & (playlists_med[i]<=k)][i]
playlists_med.join(playlists_no_na["med_popularity"]).dropna().sort_values("med_popularity", ascending = False).reset_index(drop=True)

In [197]:
def opti(objectif_min, objectif_max, playlists_no_na, playlists_med):
    """Sélectionne dans un premier temps les playlists qui correspondent aux critères puis trie la liste obtenue en fonction de la popularité."""
    for i, j, k in zip(list(playlists_med.columns), objectif_min, objectif_max):
        playlists_med[str(i) + "_filtre"] = playlists_med[(playlists_med[i]>=j) & (playlists_med[i]<=k)][i]
    return playlists_med.join(playlists_no_na["med_popularity"]).dropna().sort_values("med_popularity", ascending = False).reset_index(drop=True)

In [203]:
dict_output = {"mood":{"tres_joyeuse":[0.8, 1], "joyeuse":[0.6, 0.80], "peu_importe":[0, 1], "neutre":[0.4, 0.60], "triste":[0.2, 0.40], "tres_triste":[0, 0.20]},
"danser":{"oui":[0.75, 1], "un_peu":[0.5, 0.75], "peu_importe":[0, 1], "pas_vraiment":[0.25, 0.50], "pas_du_tout":[0, 0.25]},
"energie":{"oui":[0.75, 1], "un_peu":[0.5, 0.75], "peu_importe":[0, 1], "pas_vraiment":[0.25, 0.50], "pas_du_tout":[0, 0.25]},
"type_instruments":{"acoustiques":[0.75, 1], "majorite_acoustiques":[0.5, 0.75], "peu_importe":[0, 1], "majorite_electronique":[0.25, 0.50], "electronique":[0, 0.25]},
"paroles":{"oui":[0.75, 1], "un_peu":[0.5, 0.75], "peu_importe":[0, 1], "pas_vraiment":[0.25, 0.50], "pas_du_tout":[0, 0.25]},
"type_paroles":{"majorite_chantees":[0.5, 1], "peu_importe":[0, 1], "majorite_parlees":[0, 0.50]}}

In [204]:
dict_output["mood"]["tres_joyeuse"]

[0.8, 1]

In [208]:
def fill_dict(valence, dansability, energy, acousti, instrumentalness, speechness):
    list_type = ["mood", "danser", "energie", "type_instruments", "paroles", "type_paroles"]
    list_inputs = [valence, dansability, energy, acousti, instrumentalness, speechness]
    objectif_min = []
    objectif_max = []
    for i, j in zip(list_type, list_inputs):
        min_value, max_value = dict_output[i][j]
        objectif_min.append(min_value)
        objectif_max.append(max_value)
    return objectif_min, objectif_max       

In [8]:
playlists_no_na.iloc[:, 10:16]

,med_valence,med_danceability,med_energy,med_acousticness,med_speechiness,med_instrumentalness
1,84.600000,0.5030,0.7010,0.6085,0.19700,0.05700
2,77.260000,0.4570,0.7875,0.6135,0.07560,0.23450
3,70.865385,0.5425,0.5955,0.7050,0.14450,0.03505
4,82.000000,0.6215,0.7725,0.6860,0.13950,0.09175
5,18.720000,0.4925,0.6640,0.5955,0.13350,0.06460
...,...,...,...,...,...,...
1394,63.528571,0.4120,0.6460,0.5860,0.25750,0.05495
1395,69.020000,0.5010,0.6835,0.6320,0.09410,0.05750
1396,50.100000,0.4075,0.5695,0.5520,0.26050,0.04325
1397,69.120000,0.6535,0.6870,0.7490,0.09685,0.04775


In [4]:
import opti as opt

In [7]:
playlists_no_na.iloc[:, 10:16].reset_index(drop=True)

,med_valence,med_danceability,med_energy,med_acousticness,med_speechiness,med_instrumentalness
0,84.600000,0.5030,0.7010,0.6085,0.19700,0.05700
1,77.260000,0.4570,0.7875,0.6135,0.07560,0.23450
2,70.865385,0.5425,0.5955,0.7050,0.14450,0.03505
3,82.000000,0.6215,0.7725,0.6860,0.13950,0.09175
4,18.720000,0.4925,0.6640,0.5955,0.13350,0.06460
...,...,...,...,...,...,...
1390,63.528571,0.4120,0.6460,0.5860,0.25750,0.05495
1391,69.020000,0.5010,0.6835,0.6320,0.09410,0.05750
1392,50.100000,0.4075,0.5695,0.5520,0.26050,0.04325
1393,69.120000,0.6535,0.6870,0.7490,0.09685,0.04775


In [6]:
opt.final_opti("peu_importe", "oui", "oui", "peu_importe", "peu_importe", "peu_importe", playlists)

,med_popularity,url,med_valence,med_danceability,med_energy,med_acousticness,med_speechiness,med_instrumentalness
0,71.0,https://open.spotify.com/playlist/37i9dQZF1DX0...,0.5400,0.7550,0.8700,0.01160,0.05330,0.263000
1,54.5,https://open.spotify.com/playlist/4pL8YJw0ts5Y...,0.6485,0.7770,0.7520,0.00673,0.14600,0.037800
2,53.0,https://open.spotify.com/playlist/37i9dQZF1DXb...,0.5910,0.7620,0.7600,0.02060,0.09260,0.000000
3,49.5,https://open.spotify.com/playlist/37i9dQZF1DX7...,0.8240,0.7870,0.8120,0.17900,0.04850,0.000180
4,48.0,https://open.spotify.com/playlist/37i9dQZF1DX8...,0.6265,0.7815,0.8765,0.00398,0.05980,0.271500
5,38.0,https://open.spotify.com/playlist/2KSySxa7iudj...,0.6130,0.7630,0.8060,0.01350,0.10100,0.000001
6,35.0,https://open.spotify.com/playlist/4mM7tVGeVv4d...,0.5785,0.7710,0.7655,0.01645,0.08055,0.091000
7,34.5,https://open.spotify.com/playlist/37i9dQZF1DX3...,0.7080,0.7505,0.7635,0.01305,0.05240,0.057550
8,11.0,https://open.spotify.com/playlist/2nZndNnauO6S...,0.7300,0.8125,0.7920,0.09710,0.09785,0.014700


In [16]:
playlists_no_na[["med_popularity", "url"]]

,med_popularity,url
1,89.0,https://open.spotify.com/playlist/37i9dQZF1DXc...
2,78.5,https://open.spotify.com/playlist/37i9dQZF1DX0...
3,71.0,https://open.spotify.com/playlist/37i9dQZF1DX1...
4,82.5,https://open.spotify.com/playlist/37i9dQZF1DX1...
5,17.0,https://open.spotify.com/playlist/37i9dQZF1DX4...
...,...,...
1394,65.0,https://open.spotify.com/playlist/37i9dQZF1DX3...
1395,71.5,https://open.spotify.com/playlist/5ji4GZJpll6t...
1396,50.5,https://open.spotify.com/playlist/40VxbK9Nqccd...
1397,71.0,https://open.spotify.com/playlist/0RTz1jFo5BXG...


In [ ]:
def opti(objectif_min, objectif_max, playlists, dict_output):
    """Sélectionne dans un premier temps les playlists qui correspondent aux critères puis trie la liste obtenue en fonction de la popularité."""
    playlists_no_na = playlists.dropna()
    playlists_med = playlists_no_na.iloc[:, 10:16].reset_index(drop=True)
    for i, j, k in zip(list(playlists_med.columns), objectif_min, objectif_max):
        playlists_med[str(i) + "_filtre"] = playlists_med[(playlists_med[i]>=j) & (playlists_med[i]<=k)][i]
    return playlists_med.join(playlists_no_na["med_popularity"]).dropna().sort_values("med_popularity", ascending = False).reset_index(drop=True)

In [209]:
fill_dict("tres_joyeuse", "oui", "peu_importe", "acoustiques", "pas_vraiment", "majorite_chantees")

([0.8, 0.75, 0, 0.75, 0.25, 0.5], [1, 1, 1, 1, 0.5, 1])

# Regarde pas

In [88]:
user = 'TestAccountlab'
repo = 'spotify_app_data' 
token = "fead267f1c2020f164d8ea7467cfd1273d96eb05"

message = json.dumps({"message":"update",
                            "content": encodedStr ,
                            "sha": "6c805bf503e2d2af800dac610935c27c85b9daa8"
                            })

headers = {'Authorization': 'token ' + token}

login2 = requests.put('https://api.github.com/repos/' + user + '/' + repo + '/contents/test_data.json' , headers=headers, data=message)
                      #"https://api.github.com/repos/"+repo+"/contents/"+filename

In [64]:
login.json()['content']

'ewogICAidXNlcnMiOlsidXNlcjEiLCAidXNlcjIiCiAgICAgIAogICBdLAog\nICAicGFzc3dvcmQiOlsiMTIzNDUiLCAiNjc4OTEwIgogICAgICAKICAgXSwK\nICAgImRhdGEiOlsKICAgICAgCiAgIF0KfQo=\n'

In [89]:
login2.json()

{'content': {'name': 'test_data.json',
  'path': 'test_data.json',
  'sha': '0db1fad91762f67d818b166d94a6a2b9978e53f9',
  'size': 80,
  'url': 'https://api.github.com/repos/TestAccountlab/spotify_app_data/contents/test_data.json?ref=main',
  'html_url': 'https://github.com/TestAccountlab/spotify_app_data/blob/main/test_data.json',
  'git_url': 'https://api.github.com/repos/TestAccountlab/spotify_app_data/git/blobs/0db1fad91762f67d818b166d94a6a2b9978e53f9',
  'download_url': 'https://raw.githubusercontent.com/TestAccountlab/spotify_app_data/main/test_data.json?token=ATNTSZHX3AT7WFTLS3FF243AL5UFQ',
  'type': 'file',
  '_links': {'self': 'https://api.github.com/repos/TestAccountlab/spotify_app_data/contents/test_data.json?ref=main',
   'git': 'https://api.github.com/repos/TestAccountlab/spotify_app_data/git/blobs/0db1fad91762f67d818b166d94a6a2b9978e53f9',
   'html': 'https://github.com/TestAccountlab/spotify_app_data/blob/main/test_data.json'}},
 'commit': {'sha': '7daab2d9db4ea2101e1c3a0

In [68]:
message

str

In [38]:
import base64

encodedStr = login.json()['content']

# Standard Base64 Decoding
decodedBytes = base64.b64decode(encodedStr)
decodedStr = json(decodedBytes, "utf-8")

print(decodedStr)
test_data = decodedStr

NameError: name 'json' is not defined

In [56]:
test_change = json.loads(test_data)
test_change["data"].append("test")


In [87]:
test_change64 = base64.b64encode(str(test_change).encode("utf-8"))
encodedStr = str(test_change64, "utf-8")

In [ ]:
{
   "users":[
      
   ],
   "password":[
      
   ],
   "data":[
      
   ]
}

In [40]:
import json

In [57]:
test_change

{'users': ['user1', 'user2'],
 'password': ['12345', '678910'],
 'data': ['test']}